In [1]:
from z3 import *

from itertools import combinations
from z3 import *
from utils import *
import math
import time

In [3]:
#z3.Z3_global_param_get()
z3.Z3_get_global_param_descrs()

AttributeError: module 'z3.z3' has no attribute 'help'

In [2]:
file_name='inst13.dat'
file = open('./Instances/'+file_name, 'r')
splitted_file = file.read().split('\n')
m = int(splitted_file[0])
n = int(splitted_file[1])
cpt = list(map(int, splitted_file[2].split(' ')))
sz = list(map(int, splitted_file[3].split(' ')))
D = [list(map(int, line.strip().split(' '))) for line in splitted_file[4:(n+5)]]
print(D)

[[0, 60, 141, 22, 41, 137, 77, 48, 92, 105, 113, 103, 82, 15, 79, 24, 98, 69, 82, 30, 105, 89, 57, 94, 75, 50, 127, 16, 36, 77, 57, 70, 51, 101, 88, 38, 83, 108, 81, 124, 54, 131, 99, 70, 112, 162, 94, 64], [60, 0, 83, 82, 99, 81, 53, 106, 34, 61, 57, 43, 140, 63, 23, 56, 84, 59, 142, 88, 45, 79, 27, 34, 33, 46, 107, 58, 94, 17, 29, 26, 109, 51, 34, 30, 45, 48, 23, 64, 50, 73, 41, 26, 68, 104, 34, 36], [141, 83, 0, 129, 182, 4, 64, 189, 63, 36, 28, 126, 223, 126, 106, 139, 167, 142, 125, 171, 128, 52, 110, 117, 116, 129, 24, 125, 177, 100, 84, 71, 192, 40, 53, 103, 58, 131, 90, 147, 87, 70, 100, 71, 29, 101, 117, 77], [22, 82, 129, 0, 55, 125, 65, 60, 80, 93, 101, 125, 94, 19, 101, 46, 120, 91, 60, 42, 127, 77, 79, 116, 97, 72, 115, 24, 48, 99, 53, 58, 63, 89, 76, 52, 71, 130, 89, 146, 42, 119, 99, 58, 100, 150, 116, 52], [41, 99, 182, 55, 0, 178, 118, 11, 133, 146, 154, 118, 41, 56, 76, 43, 65, 40, 115, 41, 130, 130, 72, 125, 66, 53, 168, 57, 15, 100, 98, 111, 50, 142, 129, 79, 124, 1

In [3]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

In [4]:
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

In [5]:
def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))   
    return And(constraints)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name+'1'), at_least_k_seq(bool_vars, k, name))

In [6]:
def print_items_for_each_courier(couriers_items, items_predecessors, sz, cpt):
    # Creare un dizionario per mappare ogni corriere ai suoi pacchi
    couriers_dict = {}
    for i, j in couriers_items:
        if i not in couriers_dict:
            couriers_dict[i] = []
        couriers_dict[i].append(j)
    #print('cour_dict', couriers_dict)

    n=0
    for value in couriers_dict.values():
        n = n+len(value)
    #print(n)


    # Creare un altro dizionario per mappare ogni pacco al suo predecessore
    
    predecessors_dict = {i: j for j, i in items_predecessors}
    #print('pred_dict', predecessors_dict)
    
    # Per ogni corriere, seguire la catena di predecessori per ottenere l'ordine corretto dei pacchi
    for courier, items in couriers_dict.items():
        print(f"Corriere {courier}: ", end='')
        item = n + courier
        order = [item]
        while item in predecessors_dict and predecessors_dict[item] not in order:
            item = predecessors_dict[item]
            order.append(item)
        item = predecessors_dict[item]
        order.append(item)
        print(" -> ".join(map(str, order)), end='   ')
        print('Peso trasportato: ', end='')
        weight=0
        for i in couriers_dict[courier]:
            weight += sz[i]

        print(weight, '/', cpt[courier])

In [7]:
def max_z3(vec):
  m = vec[0]
  for value in vec[1:]:
    m = If(value > m, value, m)
  return m

In [8]:
'''n packs
   m couriers
   D matix int(n+1,m+1)
   sz array of item's sizes (len = n)
   cpt array of couriers' capacities (len = m)
'''
def mcp(n, m, D, sz, cpt):
   
   '''
      pred matrix  
   '''
   #pred[i][j] = true if the j-th item is the predecessor of the i-th item. 
   #The n+i column in the matrix is the starting point of the i-th courier. The n+i row in the matrix is the ending point of the i-th courier
   pred = [[Bool(f"pred({i})_{j}")for j in range(n+m)]for i in range(n+m)]

   #create Solver
   solv= Solver()

   '''
   # The deposit has at most n predecessor and it is predecessor of at most n items
   col_n = []
   for j in range(n):
      col_n += [pred[j][n]]
   solv.add(at_most_k_seq(pred[n],m, f"PR_{n}")) #pred row (predecessors of the deposit -- last item of each courier)
   solv.add(at_most_k_seq(col_n,m, f"PC_{n}")) #pred col (items that have deposit as predecessor -- first items of each courier)
   print(solv.check())
   '''

   #Each item/ending point has exactly one predecessor and each item/starting point is predecessor of exactly one other item. 
   #i=0 to i=n+m-1
   #j=0 to j=n+m-1 
   for i in range(n+m):
      col_i = []
      for j in range(n+m):
         col_i += [pred[j][i]]

      solv.add(exactly_one_seq(col_i, f"PC_{i}"))
      solv.add(exactly_one_seq(pred[i], f"PR_{i}"))

   
   '''
      vehicle matrix 
      col = packs
      rows = courriers 
      each rows represent the list of packs foreach courrier
   ''' 
   #create a [m x (n+m)] matrix v. v[i][j]==True if the i-th courier takes the j-th item (or starting/ending point)
   v=[[Bool(f"vehicle({i})_{j}")for j in range(n+m)]for i in range(m)]

   #each pack have to be carried by exactly one courrier
   for j in range(n):
      solv.add(exactly_one_seq([v[i][j] for i in range(m)], f"v_{j}"))

   for j in range(m):
      for i in range(m):
         if j==i:
            solv.add(v[i][n+j])
         else:
            solv.add(Not(v[i][n+j]))
   
   #Constraint coerenza courier - predecessore/successore
   for c in range(m):
      solv.add(And(  [Implies(And([ v[c][i], pred[i][j]] ) , v[c][j]) for i in range(n+m) for j in range(n+m)]  ))

   
   #Constraint peso
   for c in range(m):
      solv.add(Sum([If(v[c][i], 1, 0)*sz[i] for i in range(n)]) <= cpt[c])


   #FUNZIONA. WOW :D
   #solv.add(And( [Implies( And([pred[i][n], pred[j][n]]) , And([ Not(And(v[k][i], v[k][j])) for k in range(m)])) for i in range(n) for j in range(n) if i<j])) #Se due items hanno come predecessore il deposito allora devono essere presi da due veicoli diversi

   
   '''
      no_loops 
   '''
   no_loops = IntVector('no_loops', n)
   solv.add(And([Implies(pred[i][j], no_loops[i]>no_loops[j]) for i in range(n) for j in range(n)]))
   #solv.add(no_loops[0]==1)

   #Distance
   dist_vector = IntVector('dist_vector', m)
   for courier in range(m):
      starting_point = [If(pred[item][n+courier], 1, 0)*D[n][item] for item in range(n)]
      ending_point = [If(pred[n+courier][item], 1, 0)*D[item][n] for item in range(n)]
      mid_points = [If(And(v[courier][i], pred[i][j]), 1, 0)*D[j][i] for i in range(n) for j in range(n)]
      distance_of_this_path = Sum(starting_point + mid_points + ending_point)
      solv.add(dist_vector[courier] == distance_of_this_path)
   max_dist = Int('max_dist')
   #max_dist = max_z3(dist_vector)
   solv.add(max_dist == max_z3(dist_vector))
   
   return solv, max_dist, pred, v

In [9]:
z3.set_param(random_seed=42)
timeout = time.time() + 60*5
check_timeout = timeout-time.time()
solv, max_dist, pred, v = mcp(n,m,D,sz,cpt)

print('Total time: ', check_timeout)

first_timstamp=time.time()
checked_solv = solv.check()
second_timestamp=time.time()

print('Time used for the first search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

if checked_solv==sat:
    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    print('Time left:', timeout-time.time(), '; Distance: ', best_distance)
    print('--------------------------------------------------')

    solv.add( Not( And([pred[i][j] for i in range(n) for j in range(n+m) if best_model.evaluate(pred[i][j])] + [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])])) )
    solv.add( max_dist < best_distance)

    check_timeout = timeout-time.time()
    first_timstamp=time.time()
    checked_solv = solv.check()
    second_timestamp=time.time()
    print('Time used for search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

while time.time() < timeout and checked_solv==sat:

    print('Current time left:', timeout-time.time(), '; Best distance found so far: ', best_distance)

    check_timeout = timeout-time.time()
    solv.set('timeout', int(check_timeout*1000)) 
    
    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()
    solv.add( max_dist < best_distance)
    solv.add( Not( And([pred[i][j] for i in range(n+m) for j in range(n+m) if best_model.evaluate(pred[i][j])]  + [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])])) )
    first_timstamp=time.time()
    checked_solv = solv.check()
    second_timestamp=time.time()
    print('--------------------------------------------------')
    print('Time used for the current search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)
print('--------------------------------------------------')
print('Best Solution Found:')
    
item_pred, cour_item = [(i,j) for j in range(n+m) for i in range(n+m) if best_model.evaluate(pred[i][j])], [(i, j) for j in range(n) for i in range(m) if best_model.evaluate(v[i][j])]
print_items_for_each_courier(cour_item, item_pred, sz, cpt)
print('Minimized max distance: ', best_distance)

print('Time left:', timeout-time.time())

Total time:  300.0
Time used for the first search: 6.466022491455078 / 300.0 ; Result:  sat
Time left: 289.4579517841339 ; Distance:  1570
--------------------------------------------------


In [ ]:
z3.set_param(random_seed=42)

time_for_main_search=0.5 #between 0 and 1
starting_time=time.time()
timeout = starting_time + 60*5*time_for_main_search
check_timeout = timeout-time.time()

print('Time for main search:', check_timeout)
solv, max_dist, pred, v = mcp(n,m,D,sz,cpt)

first_timstamp=time.time()
checked_solv = solv.check()
second_timestamp=time.time()

print('Time used for the first search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

if checked_solv==sat:
    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    print('Time left:', timeout-time.time(), '; Distance: ', best_distance)
    print('--------------------------------------------------')

    solv.add( Not( And([v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])==True])) )
    solv.add( max_dist < best_distance)

    
    check_timeout = timeout-time.time()
    first_timstamp=time.time()
    checked_solv = solv.check()
    second_timestamp=time.time()
    print('Time used for search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)


while time.time() < timeout and checked_solv==sat:

    print('Current time left:', timeout-time.time(), '; Best distance found so far: ', best_distance)

    check_timeout = timeout-time.time()
    solv.set('timeout', int((timeout-time.time())*1000))

    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()
    solv.add( max_dist < best_distance)
    solv.add( Not( And( [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])==True])) )

    first_timstamp=time.time()
    checked_solv = solv.check()
    second_timestamp=time.time()
    print('--------------------------------------------------')
    print('Time used for the current search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

print('--------------------------------------------------')
print('End of first search. Time used: ', timeout-time.time(), '/', (starting_time + 60*5)*time_for_main_search)

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

timeout = time.time() + 60*(1-time_for_main_search)*5
starting_time_2=time.time()
check_timeout = timeout - starting_time_2
print('Time for optimization search:', check_timeout)

solv_2, max_dist, pred, v = mcp(n,m,D,sz,cpt)
solv_2.add(And([v[i][j] for i in range(m) for j in range(m) if best_model.evaluate(v[i][j])]))
solv_2.add(max_dist <= best_distance)

first_timstamp=time.time()
checked_solv = solv_2.check()
second_timestamp=time.time()

print('Time used for the first search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

if checked_solv==sat:
    best_model = solv_2.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    print('Time left:', timeout-time.time(), '; Distance: ', best_distance)
    print('--------------------------------------------------')

    solv_2.add( Not(And([pred[i][j] for i in range(n+m) for j in range(n+m) if best_model.evaluate(pred[i][j])]) ))
    solv_2.add( max_dist < best_distance)

    check_timeout = timeout-time.time()
    solv_2.set('timeout', int(check_timeout*1000))
    first_timstamp=time.time()
    checked_solv = solv_2.check()
    second_timestamp=time.time()
    print('Time used for search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)


while time.time() < timeout and checked_solv==sat:

    print('Current time left:', timeout-time.time(), '; Best distance found so far: ', best_distance)
    check_timeout = timeout-time.time()
    solv_2.set('timeout', int(check_timeout*1000))
    best_model = solv_2.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    solv_2.add( max_dist < best_distance)
    solv_2.add( Not( And( [pred[i][j] for i in range(n+m) for j in range(n+m) if best_model.evaluate(pred[i][j])])) )

    first_timstamp=time.time()
    checked_solv = solv_2.check()
    second_timestamp=time.time()
    print('--------------------------------------------------')
    print('Time used for the current search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

print('--------------------------------------------------')
print('Best Solution Found:')
    
item_pred, cour_item = [(i,j) for j in range(n+m) for i in range(n+m) if best_model.evaluate(pred[i][j])], [(i, j) for j in range(n) for i in range(m) if best_model.evaluate(v[i][j])]
print_items_for_each_courier(cour_item, item_pred, sz, cpt)
print('Minimized max distance: ', best_distance)

print('Time left:', timeout-time.time())


Time for main search: 150.0
Time used for the first search: 10.055383682250977 / 150.0 ; Result:  sat
Time left: 135.6693935394287 ; Distance:  1356
--------------------------------------------------
Time used for search: 112.19898700714111 / 135.65439343452454 ; Result:  sat
Current time left: 23.455406427383423 ; Best distance found so far:  1356
--------------------------------------------------
Time used for the current search: 23.469698190689087 / 23.455406427383423 ; Result:  unknown
--------------------------------------------------
End of first search. Time used:  -0.08190774917602539 / 852770915.7018057
Time for optimization search: 150.0
Time used for the first search: 32.31840753555298 / 150.0 ; Result:  sat
Time left: 113.67331457138062 ; Distance:  1312
--------------------------------------------------
Time used for search: 31.868614435195923 / 113.6029167175293 ; Result:  sat
Current time left: 81.73128938674927 ; Best distance found so far:  1312
-----------------------

: 

In [ ]:
'''
# Given formula F, find the model the maximizes the value of X 
# using at-most M iterations.
def max(F, X, M):
    s = Solver()
    s.add(F)
    last_model  = None
    i = 0
    while True:
        r = s.check()
        if r == unsat:
            if last_model != None:
                return last_model
            else:
                return unsat
        if r == unknown:
            raise Z3Exception("failed")
        last_model = s.model()
        s.add(X > last_model[X])
        i = i + 1
        if (i > M):
            raise Z3Exception("maximum not found, maximum number of iterations was reached")

x, y = Ints('x y')
F = [x > 0, x < 10, x == 2*y]
print max(F, x, 10000)
'''

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (1234820171.py, line 25)

In [ ]:

'''
   for k in range(m):
      k_courier_path=[]

      found_one_item=False
      one_item=0
      while not(found_one_item) and one_item<m:
         print(k)
         if (k, one_item) in cour_item:
            found_one_item=True
            k_courier_path.append(one_item)
            print('aa', k_courier_path)
         else:
            one_item+=1
      
      #Find successor
      if found_one_item:
         current_item = one_item
         while (current_item!=n+k): #Finchè non arrivo al deposito
            found_one_item=False
            i=0
            while not(found_one_item): #Finchè non trovo il successore
               if (i, current_item) in item_pred:
                  found_one_item=True
                  current_item=i
                  k_courier_path.append(current_item)
                  print('aaa', k_courier_path)
               else:
                  i+=1
      
         #Find Predecessor
         current_item = one_item
         while (current_item!=n+k): #Finchè non arrivo al deposito
            found_one_item=False
            i=0   
            while not(found_one_item): #Finchè non trovo il successore
               if (current_item, i) in item_pred:
                  found_one_item=True
                  current_item=i
                  k_courier_path[:0]=[current_item]
                  print('aaa', k_courier_path)
               else:
                  i+=1
         

      print(k, '-esimo corriere: ', k_courier_path)
   '''

"\nfor k in range(m):\n   k_courier_path=[]\n\n   found_one_item=False\n   one_item=0\n   while not(found_one_item) and one_item<m:\n      print(k)\n      if (k, one_item) in cour_item:\n         found_one_item=True\n         k_courier_path.append(one_item)\n         print('aa', k_courier_path)\n      else:\n         one_item+=1\n   \n   #Find successor\n   if found_one_item:\n      current_item = one_item\n      while (current_item!=n+k): #Finchè non arrivo al deposito\n         found_one_item=False\n         i=0\n         while not(found_one_item): #Finchè non trovo il successore\n            if (i, current_item) in item_pred:\n               found_one_item=True\n               current_item=i\n               k_courier_path.append(current_item)\n               print('aaa', k_courier_path)\n            else:\n               i+=1\n   \n      #Find Predecessor\n      current_item = one_item\n      while (current_item!=n+k): #Finchè non arrivo al deposito\n         found_one_item=False\n 